In [1]:
#  Necessary to allow assembly framework to run
import os
os.chdir('..')

In [2]:
import pandas as pd
import numpy as np
import inspect

import assembly
from assembly import config as asmbl_config
from assembly import db as asmbl_db
from assembly import models as asmbl_models

from lib.tweemio import twm
from lib.tweemio import similarity
from lib.tweemio import twitter

In [3]:
os.environ['ASSEMBLY_ENV'] = 'Development'
app = assembly.Assembly.init(__name__, {'default': []})

In [10]:
results = twm.calculate(app.config, screen_name='trevornoah', group='default') #, compare_to_screen_names=['realdonaldtrump'])

Evaluating similarity to realdonaldtrump with model: mdl_multinomialnbvect_v1
Evaluating similarity to jimmyfallon with model: mdl_multinomialnbvect_v1
Evaluating similarity to trevornoah with model: mdl_multinomialnbvect_v1
Evaluating similarity to billmaher with model: mdl_multinomialnbvect_v1
Evaluating similarity to stephenathome with model: mdl_multinomialnbvect_v1
Evaluating similarity to britneyspears with model: mdl_multinomialnbvect_v1
Evaluating similarity to selenagomez with model: mdl_multinomialnbvect_v1
Evaluating similarity to kimkardashian with model: mdl_multinomialnbvect_v1
Evaluating similarity to jtimberlake with model: mdl_multinomialnbvect_v1
Evaluating similarity to arianagrande with model: mdl_multinomialnbvect_v1
Evaluating similarity to theellenshow with model: mdl_multinomialnbvect_v1
Evaluating similarity to ladygaga with model: mdl_multinomialnbvect_v1
Evaluating similarity to rihanna with model: mdl_multinomialnbvect_v1
Evaluating similarity to taylorswift

In [11]:
{screen_name : np.mean(result.y_prob) for screen_name, result in results.items()}

{'realdonaldtrump': 2.0272744207938766e-07,
 'jimmyfallon': 8.435293913893682e-11,
 'trevornoah': 0.27224080456833444,
 'billmaher': 4.701467699324362e-05,
 'stephenathome': 1.3192031394302914e-08,
 'britneyspears': 1.8034018474097703e-11,
 'selenagomez': 1.0769915285670639e-11,
 'kimkardashian': 1.864278747397354e-10,
 'jtimberlake': 1.3633570325960516e-13,
 'arianagrande': 8.673472964447969e-12,
 'theellenshow': 9.35392920580554e-11,
 'ladygaga': 3.693794893881505e-09,
 'rihanna': 4.966278896967584e-12,
 'taylorswift13': 1.1532894211923138e-11,
 'justinbieber': 2.514918542447701e-14,
 'katyperry': 4.0867095196181355e-10,
 'billgates': 2.0412556035169727e-08,
 'mileycyrus': 1.3492695556535913e-14,
 'jlo': 1.5404753718814564e-10,
 'kingjames': 1.969267319830148e-09,
 'brunomars': 1.4672376455692892e-12,
 'chrissyteigen': 6.660846594382522e-09,
 'oprah': 8.47086824939767e-08,
 'drake': 2.716598253078705e-12,
 'pink': 6.489231577394058e-10,
 'liltunechi': 1.064885256145895e-14,
 'kevinha

In [ ]:
# -------

In [4]:
config = app.config

In [119]:
screen_name='realdonaldtrump'
group='default'

In [120]:
twitter_creds = config['TWITTER_API_CREDS']
tapi = twitter.TwitterApi(twitter_creds)

#  Download user timeline;  condense / merge tweets, based on settings
print(f'Downloading user timeline for: {screen_name}')
tweet_timeline = tapi.timeline(screen_name)
timeline_text = list(reversed([tli._json['full_text'] for tli in tweet_timeline]))
timeline_text = twm.condense_timeline(timeline_text, condense_factor=config['TWEET_CONDENSE_FACTOR'])

In [121]:
tweets = timeline_text

In [122]:
#  Transform tweets to vectorizer
#  Transform incoming tweets to vectorized values
vect_mdl = similarity.get_model(grp=group, model_name='transformation-countvectorizer')
x_vect = vect_mdl.transform(tweets)

In [123]:
#  Predict probability via model
screen_mdl = similarity.get_model(grp=group, model_name='realdonaldtrump')

In [124]:
screen_mdl.classes_

array([False,  True])

In [148]:
screen_mdl.classes_

array([False,  True])

In [125]:
y_pred = screen_mdl.predict(x_vect)

In [126]:
y_prob = screen_mdl.predict_proba(x_vect)

In [127]:
pred_prob_lbl[:5]

pred_prob_opp[:5]

pred_prob_true  = y_prob[:,1]
pred_prob_false = y_prob[:,0]

np.mean(y_prob[:,1])

np.mean(y_prob[:,0])

y_pred[:10]

pred_prob_true[:10]

pred_prob_false[:10]

(~y_pred * pred_prob_false)[:5]

(y_pred * pred_prob_true)[:5]

((~y_pred * (1 - pred_prob_false)) + (y_pred * pred_prob_true))[:5]

np.mean((~y_pred * (1 - pred_prob_false)) + (y_pred * pred_prob_true))

np.std((~y_pred * (1 - pred_prob_false)) + (y_pred * pred_prob_true))

# --------------------------------------------------------

labels = {}
for screen_name in config['SIMILARITY_COMPARISONS']['default']['screen_names']:
    model = similarity.get_model('default', model_name=screen_name)
    labels[screen_name]  = model.classes_



array([4.03911071e-04, 1.45133999e-17, 3.41111708e-03, 1.27098412e-10,
       9.99984482e-01])